## Price_Indicator_Analysis_MVP

Analysis of common technical analysis indicators on daily price data for different products.

### Data Collection

In [1]:
import pandas as pd
import numpy as np

import requests
import json
import quandl

import datetime

import matplotlib.pyplot as plt

import sqlite3

% matplotlib inline

The first step will be to get data from the [quandl API](https://www.quandl.com/collections/futures).  For this we use an api key generated from signing up for a free quandl account.

We create a list of the symbols for the products we are interested in, which can be found by looking at the [documentation](https://docs.quandl.com/).

Then loop through the products, read the price data into a dataframe, and attach the symbol:dataframe key:value pair to a dictionary object, after selecting the specific information we need.

In [3]:
# Get data from Quandl with api key
api_key = 'Hv95pPh1xQWzt5DFhxS7'

# Create list of symbols for the products we want data for
prod_list = ['CL', 'HO', 'NG', 'GC',
             'SI', 'AD', 'CD', 'EC',
             'BP', 'JY', 'US', 'C',
             'W', 'S', 'ES']

# Create a dict with keys as symbols and values as dataframe of price info
prod_dict = {}

# Iterate through list of prods and add data to dict
for prod in prod_list:
    # Quandl API call
    df = quandl.get('SCF/CME_{p}1_FW'.format(p=prod), authtoken=api_key)
    
    # Drop open interest column and rename Settle column
    df.drop(['Prev. Day Open Interest'], axis=1, inplace=True)
    df.rename(columns={'Open': 'open',
                      'High': 'high',
                      'Low': 'low',
                      'Settle': 'close',
                      'Volume': 'volume'}, inplace=True)
    # Add prod to dict
    prod_dict[prod] = df

Here is some summary information of the Crude Oil ('CL') dataframe

In [6]:
cl = prod_dict['CL']
cl.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2517 entries, 2005-01-03 to 2014-12-31
Data columns (total 5 columns):
open      2517 non-null float64
high      2517 non-null float64
low       2517 non-null float64
close     2517 non-null float64
volume    2517 non-null float64
dtypes: float64(5)
memory usage: 118.0 KB


In [7]:
cl.describe().T

,count,mean,std,min,25%,50%,75%,max
open,2517.0,81.796207,19.832869,34.36,65.68,82.55,96.90,145.19
high,2517.0,82.893532,19.961713,36.25,66.56,83.72,97.82,147.27
low,2517.0,80.582003,19.695067,33.55,64.65,81.25,95.56,143.22
close,2517.0,81.779758,19.855341,33.98,65.82,82.55,96.92,145.29
volume,2517.0,241235.096146,96750.320207,50.00,175510.60,242868.00,301777.00,670082.00


In [8]:
cl.head()

,open,high,low,close,volume
Date,,,,,
2005-01-03,43.10,43.10,41.25,42.12,69484.0
2005-01-04,42.18,44.13,41.85,43.91,100665.0
2005-01-05,43.90,44.00,42.60,43.39,91011.0
2005-01-06,43.39,45.75,42.80,45.56,122411.0
2005-01-07,45.50,46.10,44.75,45.43,99725.0
